### Análisis de Datos del Mercado de Criptomonedas con Binance API

- En la era digital actual, el mercado de criptomonedas ha emergido como un campo fascinante y dinámico, ofreciendo oportunidades únicas para el análisis de datos y la comprensión de las tendencias del mercado financiero. Nuestro proyecto se centra en explorar este sector vibrante utilizando una de las herramientas más poderosas en el mundo de las criptomonedas: la API de Binance.

- Binance, siendo una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial, proporciona una API rica y robusta, permitiendo a los usuarios acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye, pero no se limita a, datos históricos de precios, detalles de operaciones individuales, volúmenes de trading, y mucho más.

- El objetivo principal de este proyecto es aprovechar estos datos para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder en el mercado. Nos centraremos en extraer datos históricos de precios y operaciones de BTC comparados con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

- Al final de este proyecto, esperamos tener un análisis detallado y multifacético del mercado de Bitcoin, brindando un panorama claro no solo para los entusiastas de las criptomonedas, sino también para aquellos interesados en las finanzas y la tecnología. Este desafío es una excelente oportunidad para sumergirnos en el mundo del análisis de datos financieros y abrir la puerta a futuras investigaciones y proyectos en este apasionante campo.

In [1]:
# paquetes necesarios
import requests
import pandas as pd
import hmac
import hashlib
import time
import matplotlib.pyplot as plt

In [13]:
api_key = '4BfPXEo2L55fb4WFe6EaNVVCl3JBuCyrEn75lM3y6R1Ut7HiLqTKBfBe5wquc85d'
api_secret = '9MZFj7DhnZTOViXtjMq7oupWg7aaN4bx7Xc7qb4ezPL0MOQt0IXKHLylYngFFPA0'
api_limit = 1000

# Función para firmar los parámetros
def sign_request(api_secret, params):
    query_string = '&'.join(["{}={}".format(d, params[d]) for d in params])
    signature = hmac.new(api_secret.encode(), query_string.encode(), hashlib.sha256).hexdigest()
    return signature

# Obtener operaciones históricas
def get_historical_trades(api_key, api_secret):
    url = "https://api.binance.com/api/v3/historicalTrades"
    params = {
        'symbol': 'BTCUSDT',
        'limit': api_limit
    }
    headers = {'X-MBX-APIKEY': api_key}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error en la solicitud de operaciones históricas: Código de estado {response.status_code}")
        print(response.text)
        return None
    else:
        return pd.DataFrame(response.json())

# Obtener precios históricos
def get_historical_prices(api_key, api_secret):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        'symbol': 'BTCUSDT',
        'interval': '1h',
        'limit': api_limit
    }
    headers = {'X-MBX-APIKEY': api_key}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error en la solicitud de precios históricos: Código de estado {response.status_code}")
        print(response.text)
        return None
    else:
        data = response.json()
        columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore']
        df = pd.DataFrame(data, columns=columns)
        df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
        df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
        return df

df_trades = get_historical_trades(api_key, api_secret)
df_prices = get_historical_prices(api_key, api_secret)

## Unificar los 2 dataFrames en uno

In [3]:
df_trades['Time Interval'] = pd.to_datetime(df_trades['time'], unit='ms').dt.floor('H')

# conbinacion de dataframes
# Usamos un merge left para mantener todas las filas de df_trades y agregar la información correspondiente de df_prices
combined_df = pd.merge(df_trades, df_prices, left_on='Time Interval', right_on='Open Time', how='left')
combined_df.head()

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,3393187251,41838.23000000,0.00021000,8.78602830,1706474204237,True,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41861.10000000,41681.72000000,41861.10000000,733.14339000,2024-01-28 20:59:59.999,30626535.62206610,28214,407.45438000,17024060.95565210,0
1,3393187252,41838.22000000,0.00019000,7.94926180,1706474204237,True,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41861.10000000,41681.72000000,41861.10000000,733.14339000,2024-01-28 20:59:59.999,30626535.62206610,28214,407.45438000,17024060.95565210,0
2,3393187253,41837.99000000,0.00021000,8.78597790,1706474204237,True,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41861.10000000,41681.72000000,41861.10000000,733.14339000,2024-01-28 20:59:59.999,30626535.62206610,28214,407.45438000,17024060.95565210,0
3,3393187254,41837.75000000,0.00021000,8.78592750,1706474204265,True,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41861.10000000,41681.72000000,41861.10000000,733.14339000,2024-01-28 20:59:59.999,30626535.62206610,28214,407.45438000,17024060.95565210,0
4,3393187255,41837.75000000,0.00019000,7.94917250,1706474204269,True,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41861.10000000,41681.72000000,41861.10000000,733.14339000,2024-01-28 20:59:59.999,30626535.62206610,28214,407.45438000,17024060.95565210,0


## Cambiar columnas de tipo bool a valores de cero o uno

In [4]:
# Lista de columnas de tipo bool
bool_columns = combined_df.select_dtypes(include=['bool']).columns

# Convertir cada columna bool a int
for column in bool_columns:
    combined_df[column] = combined_df[column].astype(int)

# Verificar los cambios
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            1000 non-null   int64         
 1   price                         1000 non-null   object        
 2   qty                           1000 non-null   object        
 3   quoteQty                      1000 non-null   object        
 4   time                          1000 non-null   int64         
 5   isBuyerMaker                  1000 non-null   int32         
 6   isBestMatch                   1000 non-null   int32         
 7   Time Interval                 1000 non-null   datetime64[ns]
 8   Open Time                     1000 non-null   datetime64[ns]
 9   Open                          1000 non-null   object        
 10  High                          1000 non-null   object        
 11  Low                           1

## Cambiar columnas del tipo Object a Float(float64)

In [5]:
object_columns_to_convert = [
    'price', 'qty', 'quoteQty', 'Open', 'High', 'Low', 
    'Close', 'Volume', 'Quote Asset Volume', 
    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
]

# Convertir cada columna seleccionada a float
for column in object_columns_to_convert:
    combined_df[column] = pd.to_numeric(combined_df[column], errors='coerce')

# Verificar los cambios
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            1000 non-null   int64         
 1   price                         1000 non-null   float64       
 2   qty                           1000 non-null   float64       
 3   quoteQty                      1000 non-null   float64       
 4   time                          1000 non-null   int64         
 5   isBuyerMaker                  1000 non-null   int32         
 6   isBestMatch                   1000 non-null   int32         
 7   Time Interval                 1000 non-null   datetime64[ns]
 8   Open Time                     1000 non-null   datetime64[ns]
 9   Open                          1000 non-null   float64       
 10  High                          1000 non-null   float64       
 11  Low                           1

## Cambiar columnas del tipo fecha a milisegundos

In [6]:
# Lista de columnas de tipo datetime64[ns] que deseas convertir a milisegundos
datetime_columns = ['Time Interval', 'Open Time', 'Close Time']

# Convertir cada columna datetime a milisegundos
for column in datetime_columns:
    combined_df[column] = (combined_df[column].astype('int64') // 10**6)

# Verificar los cambios
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            1000 non-null   int64  
 1   price                         1000 non-null   float64
 2   qty                           1000 non-null   float64
 3   quoteQty                      1000 non-null   float64
 4   time                          1000 non-null   int64  
 5   isBuyerMaker                  1000 non-null   int32  
 6   isBestMatch                   1000 non-null   int32  
 7   Time Interval                 1000 non-null   int64  
 8   Open Time                     1000 non-null   int64  
 9   Open                          1000 non-null   float64
 10  High                          1000 non-null   float64
 11  Low                           1000 non-null   float64
 12  Close                         1000 non-null   float64
 13  Volu

In [7]:
combined_df.sample(7)

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
195,3393187446,41833.43,0.00182,76.136843,1706474219999,1,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
385,3393187636,41843.75,0.00116,48.538750,1706474240973,0,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
423,3393187674,41843.75,0.00048,20.085000,1706474248954,0,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
714,3393187965,41850.00,0.00127,53.149500,1706474280006,0,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
965,3393188216,41859.99,0.00353,147.765765,1706474302785,1,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
742,3393187993,41850.00,0.00727,304.249500,1706474284696,0,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
296,3393187547,41837.27,0.00021,8.785827,1706474234212,0,1,1706472000000,1706472000000,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0


In [8]:
# Cambiar el índice a la columna 'id'
combined_df.set_index('id', inplace=True)

In [18]:
final_DS = combined_df

In [19]:
final_DS.sample(7)

,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
id,,,,,,,,,,,,,,,,,,,
3393187606,41842.81,0.00240,100.422744,1706474238912,0,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
3393187253,41837.99,0.00021,8.785978,1706474204237,1,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
3393188062,41852.65,0.00003,1.255580,1706474294792,0,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
3393188066,41852.83,0.00014,5.859396,1706474294792,0,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
3393187449,41833.43,0.00141,58.985136,1706474220002,1,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
3393187451,41833.44,0.00240,100.400256,1706474220004,0,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0
3393187696,41844.28,0.00020,8.368856,1706474248954,0,1,1706472000000,2024-01-28 20:00:00,41785.91,41861.1,41681.72,41861.1,733.14339,1706475599999,3.062654e+07,28214,407.45438,1.702406e+07,0


In [20]:
final_DS.corr()

,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
price,1.000000,0.023474,0.023510,0.952714,-0.191495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
qty,0.023474,1.000000,1.000000,0.018565,-0.084627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quoteQty,0.023510,1.000000,1.000000,0.018597,-0.084634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,0.952714,0.018565,0.018597,1.000000,-0.250135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isBuyerMaker,-0.191495,-0.084627,-0.084634,-0.250135,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isBestMatch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Time Interval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Open Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
High,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
